* author: Yunlu Huang
* email: yunluh@andrew.cmu.edu

<FONT COLOR = 'RED'>以下内容包含处理数据的代码 & 思路详细解释（具体看每个部分的 `处理思路`）</FONT>

# 目录

* [STEP 0. 读取训练数据 (Development Data)](#read)
* [STEP 1. 样本处理：缺失值、异常值、重复值](#step1-1)
> * [STEP 1-1. 处理缺失值](#step1-1)
> * [STEP 1-2. 处理异常值、重复值](#step1-2)
* [STEP 2. 变量处理：日期处理、数值标准化、分类变量处理](#step2-1)
> * [STEP 2-1. 日期处理](#step2-1)
> * [STEP 2-2. 数值标准化](#step2-2)
> * [STEP 2-3. 分类变量处理](#step2-3)
* [其他：检验数据 (Assessment Data) 清洗](#assessment)
* [STEP 3. 数据不均衡：综合采样](#resample)
* [STEP 4. 特征选择 & 特征提取](#features)
* [STEP 5. 建模](#models)
> * [STEP 5-1. Logistic Regression](#lr)
> * [STEP 5-2. SVM](#svm)
> * [STEP 5-3. Random Forest](#rf)
* [STEP 6. 模型选择](#selection)
* [STEP 7. 分类预测](#prediction)

In [2]:
import xlrd
import math
import sklearn
import numpy as np
import pandas as pd
from collections import Counter
from pyxlsb import convert_date
from pyxlsb import open_workbook
from imblearn.combine import SMOTEENN
from sklearn.utils import resample 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score

<a name="read"></a>
# STEP 0. 读取数据

In [2]:
# read the data
wb = open_workbook('development_sample.xlsb')
sheet = wb.get_sheet('DEV')
data = []
for i, row in enumerate(sheet.rows()):
    row_cells = []
    for j, cell in enumerate(row):
        row_cells.append(convert_date(cell.v) if j == 4 and i > 0 else cell.v)
    data.append(row_cells)
wb.close()
# generate the dataframe based on the data
df = pd.DataFrame(data, columns=data[0])
df = df.iloc[1:]

In [3]:
# copy the dataframe
df_clean = df.copy()

In [6]:
df_clean.head()

,TARGET,ID2,LOYALTY_STATUS,WEEK_DAY,DDATE,SOCIAL_STATUS,HOUR,AGE_RANGE,EDUCATION,COMPFIELD,...,DIF_CELLTEL_ALLPERIOD,DIF_INN_BY_CELLTEL_ALLPERIOD,DIF_INN_BY_CELLTEL_LAST_30_,DIF_CNT_SPEC_ALLPERIOD,DIF_INN_CNT_ALLPERIOD,DIF_INN_BY_3PERTEL_ALLPERIOD,DIF_INN_BY_HOMETEL_ALLPERIOD,FIRST_DEALNO_LAG,DIF_INN_BY_COMPTEL_LAST_180_,DIF_INN_BY_COMPTEL_LAST_30_
1,0,1,0,1,2004-12-01 00:00:00,wage worker,11,40-44,upper-secondary,02,...,0,0,0,0,0,0,1,None,0,0
2,1,2,0,6,2004-12-01 00:00:00,self-emplyed,13-17,>54,upper-secondary,01,...,0,0,0,0,0,1,0,None,0,0
3,0,3,0,1,2004-12-01 00:00:00,wage worker,13-17,35-39,upper-secondary,02,...,0,0,0,0,0,0,0,None,0,0
4,0,4,0,5,2004-12-01 00:00:00,self-emplyed,11,26-29,secondary,12,...,0,0,0,0,0,0,2,None,0,0
5,0,5,0,4,2004-12-01 00:00:00,self-emplyed,>18,40-44,upper-secondary,04,...,0,0,0,3,0,0,1,None,0,0


<a name="step1-1"></a>
# STEP 1-1. 处理缺失值

In [7]:
# count the missing values number in each coloumn
for i, col_name in enumerate(data[0]):
    count = df_clean[col_name].isnull().sum().sum()
    if count > 0:
        print("col#{} \"{}\" : {}".format(i+1, col_name, count))
        
# The output is:
# col#1 "TARGET" : 1
# col#10 "COMPFIELD" : 445
# col#12 "COMPSTAFFQNTY" : 1
# col#13 "APPWPERIOD_R2" : 1062
# col#14 "APPCHILDQNTY" : 12127
# col#64 "FIRST_DEALNO_LAG" : 245506

col#1 "TARGET" : 1
col#10 "COMPFIELD" : 445
col#12 "COMPSTAFFQNTY" : 1
col#13 "APPWPERIOD_R2" : 1062
col#14 "APPCHILDQNTY" : 12127
col#64 "FIRST_DEALNO_LAG" : 245506


### 处理思路：

因为以前没有太多处理缺失值的经验，经过查阅，认为可行的缺失值处理方式如下（摘自知乎”机器学习如何处理缺失值？“ [https://www.zhihu.com/question/26639110] ）：
> 1. 平均值、中值、分位数、众数、随机值。缺陷在于：人为增加了噪声
> 2. 其他变量做预测模型来算出缺失变量。缺陷在于：如果其他变量和缺失变量无关，则预测的结果无意义；如果预测结果相当准确，则又说明这个变量是没必要加入建模
> 3. 最精确的做法，把变量映射到高维空间。比如性别，有男、女、缺失三种情况，则映射成3个变量：是否男、是否女、是否缺失。好处：完整保留了原始数据的全部信息

综合以上所得数据，发现缺失值全部为分类变量（没有数值型变量），决定处理以下方式：
> 1. 缺失值数量为1的 -> 删除
> 2. 其他缺失值 -> 变量映射到高维空间

In [8]:
# delete the rows with missing values count = 1
df_clean = df_clean.dropna(subset = ['TARGET', 'COMPSTAFFQNTY'])

In [11]:
# replace the missing value into 'isMissing'
df_clean['COMPFIELD'] = df_clean['COMPFIELD'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)
df_clean['APPWPERIOD_R2'] = df_clean['APPWPERIOD_R2'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)
df_clean['APPCHILDQNTY'] = df_clean['APPCHILDQNTY'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)
df_clean['FIRST_DEALNO_LAG'] = df_clean['FIRST_DEALNO_LAG'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)

<a name="step1-2"></a>
# STEP 1-2. 处理异常值、重复值

In [12]:
# check the outliers
Counter(list(df_clean['APPCHILDQNTY'])).most_common()

[('1-3', 149698),
 ('0', 134901),
 ('isMissing', 12127),
 ('>3', 1143),
 ('8', 9),
 ('10', 8),
 ('9', 6),
 ('11', 2),
 ('13', 1)]

In [13]:
# check whether there exsits duplicated rows
count = 0
for i, result in enumerate(df_clean.duplicated()):
    if result == True:
        count += 1
print(count)
# there's no duplicated rows

0


### 处理思路：

异常值：通过对不同列的数据分布进行统计分析，发现 `APPCHILDQNTY` 这一列比较奇怪——已经有`>3`的类别了，还是出现了`8`，`9`，`10`，`11` 和 `13` 的值，而且总数为26个。此外，`HOUR` 仅有一条数据值为 1，`APPWPERIOD_R2` 一条数据值为 0.5，`DIF_INN_BY_CELLTEL_ALLPERIOD`一条数据值为 7.0。
> * 把这些异常值归类到 `>3` 中
> * 删除其他异常值的数据

重复值：数据里没有重复值

In [15]:
df_clean['APPCHILDQNTY'] = df_clean['APPCHILDQNTY'].apply(lambda x: '>3' if x in ['8','9','10','11','13'] else x)
# check the outliers
Counter(list(df_clean['APPCHILDQNTY'])).most_common()

[('1-3', 149698), ('0', 134901), ('isMissing', 12127), ('>3', 1169)]

<a name="step2-1"></a>
# STEP 2-1. 日期处理

### 处理思路：

训练数据中，日期范围在2014年8月~12月，具体日子均为每月1日。仅有月份提供了实际信息。
> * 仅把月份提取出来，删除年份、日子信息

In [17]:
df_clean['DDATE'] = df_clean['DDATE'].apply(lambda x: x.month)
df_clean.head()

,TARGET,ID2,LOYALTY_STATUS,WEEK_DAY,DDATE,SOCIAL_STATUS,HOUR,AGE_RANGE,EDUCATION,COMPFIELD,...,DIF_CELLTEL_ALLPERIOD,DIF_INN_BY_CELLTEL_ALLPERIOD,DIF_INN_BY_CELLTEL_LAST_30_,DIF_CNT_SPEC_ALLPERIOD,DIF_INN_CNT_ALLPERIOD,DIF_INN_BY_3PERTEL_ALLPERIOD,DIF_INN_BY_HOMETEL_ALLPERIOD,FIRST_DEALNO_LAG,DIF_INN_BY_COMPTEL_LAST_180_,DIF_INN_BY_COMPTEL_LAST_30_
1,0,1,0,1,12,wage worker,11,40-44,upper-secondary,02,...,0,0,0,0,0,0,1,isMissing,0,0
2,1,2,0,6,12,self-emplyed,13-17,>54,upper-secondary,01,...,0,0,0,0,0,1,0,isMissing,0,0
3,0,3,0,1,12,wage worker,13-17,35-39,upper-secondary,02,...,0,0,0,0,0,0,0,isMissing,0,0
4,0,4,0,5,12,self-emplyed,11,26-29,secondary,12,...,0,0,0,0,0,0,2,isMissing,0,0
5,0,5,0,4,12,self-emplyed,>18,40-44,upper-secondary,04,...,0,0,0,3,0,0,1,isMissing,0,0


In [18]:
Counter(list(df_clean['DDATE']))

Counter({8: 48249, 9: 49071, 10: 50126, 11: 56163, 12: 94286})

<a name="step2-2"></a>
# STEP 2-2. 数值标准化

### 处理思路：

在所有变量中，仅 `WEEK_DAY`和`DDATE`为数值，其余均为分类变量。而这两个数值型变量周期并不为10（`WEEK_DAY` 周期为7， `DDATE`周期为12）。
> * Time is a cyclic variable. To encode this as a numerical feature, we can use a sine/cosine transformation. Suppose we have a feature of value f that ranges from 0 to N. Then, the sine and cosine transformation would be $\sin\left(2\pi \frac{f}{N}\right)$ and $\cos\left(2\pi \frac{f}{N}\right)$. For example, the sine transformation of 6 hours would be $\sin\left(2\pi \frac{6}{24}\right)$, since there are 24 hours in a cycle.

In [20]:
df_clean['WEEK_DAY'] = df_clean['WEEK_DAY'].astype(int)

sin_wd = df_clean['WEEK_DAY'].apply(lambda x: math.sin(2*np.pi*x/float(7)))
cos_wd = df_clean['WEEK_DAY'].apply(lambda x: math.cos(2*np.pi*x/float(7)))
sin_m = df_clean['DDATE'].apply(lambda x: math.sin(2*np.pi*x/float(12)))
cos_m = df_clean['DDATE'].apply(lambda x: math.cos(2*np.pi*x/float(12)))

df_clean = df_clean.assign(sin_week_day = sin_wd, 
                           cos_week_day = cos_wd, 
                           sin_month = sin_m, 
                           cos_month = cos_m).drop(['WEEK_DAY', 'DDATE'], axis=1)
df_clean.head()

,TARGET,ID2,LOYALTY_STATUS,SOCIAL_STATUS,HOUR,AGE_RANGE,EDUCATION,COMPFIELD,APPFAMILYSTATUS,COMPSTAFFQNTY,...,DIF_INN_CNT_ALLPERIOD,DIF_INN_BY_3PERTEL_ALLPERIOD,DIF_INN_BY_HOMETEL_ALLPERIOD,FIRST_DEALNO_LAG,DIF_INN_BY_COMPTEL_LAST_180_,DIF_INN_BY_COMPTEL_LAST_30_,cos_month,cos_week_day,sin_month,sin_week_day
1,0,1,0,wage worker,11,40-44,upper-secondary,02,1,>50,...,0,0,1,isMissing,0,0,1.0,0.623490,-2.449294e-16,0.781831
2,1,2,0,self-emplyed,13-17,>54,upper-secondary,01,2,1,...,0,1,0,isMissing,0,0,1.0,0.623490,-2.449294e-16,-0.781831
3,0,3,0,wage worker,13-17,35-39,upper-secondary,02,4,1,...,0,0,0,isMissing,0,0,1.0,0.623490,-2.449294e-16,0.781831
4,0,4,0,self-emplyed,11,26-29,secondary,12,1,1,...,0,0,2,isMissing,0,0,1.0,-0.222521,-2.449294e-16,-0.974928
5,0,5,0,self-emplyed,>18,40-44,upper-secondary,04,1,1,...,0,0,1,isMissing,0,0,1.0,-0.900969,-2.449294e-16,-0.433884


<a name="step2-3"></a>
# STEP 2-3. 分类变量处理

In [21]:
columns_list = df_clean.columns
binary_class_cols = []
for col in columns_list:
    tmp = df_clean[col].unique()
    if len(tmp) == 2:
        binary_class_cols.append(col)
        print("{}  {}".format(col,tmp))

TARGET  [0.0 1.0]
LOYALTY_STATUS  [0.0 1.0]
CONTACT_IN_BL  ['0' '1']
DIF_CELLTEL_LAST_180DAY  ['0' '>0']
DIF_CHILDQNTY_LAST_180DAY  ['0' '>0']
DIF_COMPTEL_LAST_180DAY_G2  ['0' '>0']
DIF_CONT_SS_LAST_180DAY  ['0' '>0']
DIF_EDUCATION_LAST_180DAY  ['0' '>0']
DIF_FAMSTATUS_LAST_180DAY  ['0' '>0']
DIF_FAMSTATUS_LAST_90DAY  ['0' '>0']
DIF_FMQNTY_LAST_180DAY  ['0' '>0']
DIF_INN_BY_CELLTEL_LAST_180DAY  ['0' '>0']
DIF_INN_BY_CELLTEL_LAST_30DAY  ['0' '>0']
DIF_INN_BY_CONT_SS_LAST_180_  ['0' '>0']
DIF_INN_BY_HOMETL_LST_30DAYSG2  ['0' '>0']
DIF_MSGREGION_LAST_60_80  ['0' '<180']
DIF_MSGREGION_LAST_60DAY  ['0' '>0']
DIF_REGTEL_LAST_180DAY  ['0' '>0']
DIF_SOCSTATUS_LAST_180DAY_G2  ['0' '>0']
DIF_TP_LAST_90DAY_G2  ['0' '>0']
DIF_3PESTEL_LAST_180DAY_G2  ['0' '>0']
DIF_INN_BY_CELLTEL_LAST_30_  ['0' '>0']


### 处理思路：

在所有变量中，除 `WEEK_DAY`和`DDATE`，其他全部为分类变量。从上可得所有变量中，类别范围仅为2种的变量。其中，`TARGET` 和 `LOYALTY_STATUS` ，`CONTACT_IN_BL` 的取值为 0 或 1，其他变量取值范围为：0 或 >0， 0 或 <180 。
> 1. 取值为 '0'/'1' 的分类变量保持不变，类型变为数值型
> 2. 取值为 '0'/'>0' 和 '0'/'<180' 的分类变量将不为0的取值改为1
> 3. 取值范围大于2类的分类变量，生成one-hot vectors（id化）

In [22]:
#1. change the data type
df_clean['TARGET'] = df_clean['TARGET'].astype(int)
df_clean['LOYALTY_STATUS'] = df_clean['LOYALTY_STATUS'].astype(int)
df_clean['CONTACT_IN_BL'] = df_clean['CONTACT_IN_BL'].astype(int)

In [23]:
#2. change the values
for col in binary_class_cols[3:]:
    df_clean[col] = df_clean[col].apply(lambda x: 0 if x=='0' else 1)

In [24]:
# 3. one hot vectors
other_class_cols = [c for c in columns_list if c not in binary_class_cols]
other_class_cols = other_class_cols[1:-4]

for i, col in enumerate(other_class_cols):
    if i == 0:
        id_cols = pd.get_dummies(df_clean[col], prefix = col).astype(int)
    else:
        id_cols = id_cols.join(pd.get_dummies(df_clean[col], prefix = col).astype(int))
        
# check whether the columns number is correct
print(len(id_cols.columns) == sum([len(df_clean[col].unique()) for col in other_class_cols]))

# join the table
df_clean = df_clean.join(id_cols).drop(other_class_cols, axis=1)
print(len(df_clean.columns))

True
247


In [25]:
# SUMMARY:
# the columns which are not modified in STEP 2-31
print([col for col in columns_list if col not in other_class_cols and col not in binary_class_cols])

['ID2', 'cos_month', 'cos_week_day', 'sin_month', 'sin_week_day']


<a name="assessment"></a>
# 其他： Assessment Data 处理

以上两个部分就是对training data的简单清洗，现在按照之前处理training data的顺序处理assessment data:

#### STEP 0 读取数据

In [27]:
# read the data
validation_wb = open_workbook('assessment_sample.xlsb')
validation_sheet = validation_wb.get_sheet('TEST')
validation_data = []
for i, row in enumerate(validation_sheet.rows()):
    row_cells = []
    for j, cell in enumerate(row):
        #'DDATE' is the 3rd column now
        row_cells.append(convert_date(cell.v) if j == 3 and i > 0 else cell.v)
    validation_data.append(row_cells)
validation_wb.close()
# generate the dataframe based on the data
validation_df = pd.DataFrame(validation_data, columns = validation_data[0])
validation_df = validation_df.iloc[1:]

In [28]:
# co### STEP 0py the dataframe
validation_df_clean = validation_df.copy()

#### STEP 1-1 缺失值

In [29]:
# count the missing values number in each coloumn
# the columns with missing values are the same as those in the training data
for i, col_name in enumerate(validation_data[0]):
    count = validation_df_clean[col_name].isnull().sum().sum()
    if count > 0:
        print("col#{} \"{}\" : {}".format(i+1, col_name, count))

col#9 "COMPFIELD" : 418
col#12 "APPWPERIOD_R2" : 655
col#13 "APPCHILDQNTY" : 7200
col#63 "FIRST_DEALNO_LAG" : 153387


In [30]:
# replace the missing value into 'isMissing'
validation_df_clean['COMPFIELD'] = validation_df_clean['COMPFIELD'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)
validation_df_clean['APPWPERIOD_R2'] = validation_df_clean['APPWPERIOD_R2'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)
validation_df_clean['APPCHILDQNTY'] = validation_df_clean['APPCHILDQNTY'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)
validation_df_clean['FIRST_DEALNO_LAG'] = validation_df_clean['FIRST_DEALNO_LAG'].apply(lambda x: 'isMissing' if x is np.NaN or x is None else x)

#### STEP 1-2 异常值、重复值

In [31]:
# check the outliers
# the columns with outliers are the same as those in the training data
Counter(list(validation_df_clean['APPCHILDQNTY'])).most_common()
#[('### STEP 01-3', 149698), ('0', 134901), ('isMissing', 12127), ('>3', 1169)]

[('1-3', 90285),
 ('0', 83292),
 ('isMissing', 7200),
 ('>3', 744),
 ('8', 7),
 ('10', 7),
 ('11', 2),
 ('9', 2),
 ('13', 1),
 ('12', 1)]

In [32]:
validation_df_clean['APPCHILDQNTY'] = validation_df_clean['APPCHILDQNTY'].apply(lambda x: '>3' if x in ['8','9','10','11','12','13'] else x)
Counter(list(validation_df_clean['APPCHILDQNTY'])).most_common()

[('1-3', 90285), ('0', 83292), ('isMissing', 7200), ('>3', 764)]

In [33]:
# check whether there exsits duplicated rows
count = 0
for i, result in enumerate(validation_df_clean.duplicated()):
    if result == True:
        count += 1
print(count)

0


#### STEP 2-1 处理日期

In [34]:
validation_df_clean['DDATE'] = validation_df_clean['DDATE'].apply(lambda x: x.month)
validation_df_clean.head()

,ID2,LOYALTY_STATUS,WEEK_DAY,DDATE,SOCIAL_STATUS,HOUR,AGE_RANGE,EDUCATION,COMPFIELD,APPFAMILYSTATUS,...,DIF_CELLTEL_ALLPERIOD,DIF_INN_BY_CELLTEL_ALLPERIOD,DIF_INN_BY_CELLTEL_LAST_30_,DIF_CNT_SPEC_ALLPERIOD,DIF_INN_CNT_ALLPERIOD,DIF_INN_BY_3PERTEL_ALLPERIOD,DIF_INN_BY_HOMETEL_ALLPERIOD,FIRST_DEALNO_LAG,DIF_INN_BY_COMPTEL_LAST_180_,DIF_INN_BY_COMPTEL_LAST_30_
1,297898,1,1,12,pensioner,13-17,50-54,higher,12,1,...,0,0,0,2,0,1,2,isMissing,0,0
2,297899,1,2,12,wage worker,>18,50-54,upper-secondary,02,2,...,1,0,0,4,0,0,1,isMissing,0,0
3,297900,0,6,12,pensioner,12,>54,higher,12,1,...,0,1,0,0,1,1,1,isMissing,0,0
4,297901,0,7,12,wage worker,13-17,50-54,upper-secondary,02,1,...,0,0,0,0,0,0,0,isMissing,1,0
5,297902,1,7,12,wage worker,13-17,35-39,upper-secondary,08,1,...,0,0,0,5,0,0,0,isMissing,0,0


In [35]:
Counter(list(validation_df_clean['DDATE']))

Counter({1: 55811,
         2: 51256,
         8: 12062,
         9: 12268,
         10: 12531,
         11: 14041,
         12: 23572})

#### STEP 2-2 数值标准化

In [36]:
validation_df_clean['WEEK_DAY'] = validation_df_clean['WEEK_DAY'].astype(int)

validation_sin_wd = validation_df_clean['WEEK_DAY'].apply(lambda x: math.sin(2*np.pi*x/float(7)))
validation_cos_wd = validation_df_clean['WEEK_DAY'].apply(lambda x: math.cos(2*np.pi*x/float(7)))
validation_sin_m = validation_df_clean['DDATE'].apply(lambda x: math.sin(2*np.pi*x/float(12)))
validation_cos_m = validation_df_clean['DDATE'].apply(lambda x: math.cos(2*np.pi*x/float(12)))

validation_df_clean = validation_df_clean.assign(sin_week_day = validation_sin_wd, 
                                                 cos_week_day = validation_cos_wd, 
                                                 sin_month = validation_sin_m, 
                                                 cos_month = validation_cos_m).drop(['WEEK_DAY', 'DDATE'], axis=1)
validation_df_clean.head()

,ID2,LOYALTY_STATUS,SOCIAL_STATUS,HOUR,AGE_RANGE,EDUCATION,COMPFIELD,APPFAMILYSTATUS,COMPSTAFFQNTY,APPWPERIOD_R2,...,DIF_INN_CNT_ALLPERIOD,DIF_INN_BY_3PERTEL_ALLPERIOD,DIF_INN_BY_HOMETEL_ALLPERIOD,FIRST_DEALNO_LAG,DIF_INN_BY_COMPTEL_LAST_180_,DIF_INN_BY_COMPTEL_LAST_30_,cos_month,cos_week_day,sin_month,sin_week_day
1,297898,1,pensioner,13-17,50-54,higher,12,1,1,>5,...,0,1,2,isMissing,0,0,1.0,0.623490,-2.449294e-16,7.818315e-01
2,297899,1,wage worker,>18,50-54,upper-secondary,02,2,>50,>5,...,0,0,1,isMissing,0,0,1.0,-0.222521,-2.449294e-16,9.749279e-01
3,297900,0,pensioner,12,>54,higher,12,1,1,>5,...,1,1,1,isMissing,0,0,1.0,0.623490,-2.449294e-16,-7.818315e-01
4,297901,0,wage worker,13-17,50-54,upper-secondary,02,1,>50,>5,...,0,0,0,isMissing,1,0,1.0,1.000000,-2.449294e-16,-2.449294e-16
5,297902,1,wage worker,13-17,35-39,upper-secondary,08,1,>50,>5,...,0,0,0,isMissing,0,0,1.0,1.000000,-2.449294e-16,-2.449294e-16


#### STEP 2-3 分类变量处理

In [37]:
validation_columns_list = validation_df_clean.columns
validation_binary_class_cols = []
for col in validation_columns_list:
    tmp = validation_df_clean[col].unique()
    if len(tmp) == 2:
        print("{}  {}".format(col,tmp))
        validation_binary_class_cols.append(col)

LOYALTY_STATUS  [1.0 0.0]
CONTACT_IN_BL  ['0' '1']
DIF_CELLTEL_LAST_180DAY  ['0' '>0']
DIF_CHILDQNTY_LAST_180DAY  ['0' '>0']
DIF_COMPTEL_LAST_180DAY_G2  ['0' '>0']
DIF_CONT_SS_LAST_180DAY  ['0' '>0']
DIF_EDUCATION_LAST_180DAY  ['0' '>0']
DIF_FAMSTATUS_LAST_180DAY  ['0' '>0']
DIF_FAMSTATUS_LAST_90DAY  ['0' '>0']
DIF_FMQNTY_LAST_180DAY  ['0' '>0']
DIF_INN_BY_CELLTEL_LAST_180DAY  ['0' '>0']
DIF_INN_BY_CELLTEL_LAST_30DAY  ['0' '>0']
DIF_INN_BY_CONT_SS_LAST_180_  ['0' '>0']
DIF_INN_BY_HOMETL_LST_30DAYSG2  ['0' '>0']
DIF_MSGREGION_LAST_60_80  ['0' '<180']
DIF_MSGREGION_LAST_60DAY  ['0' '>0']
DIF_REGTEL_LAST_180DAY  ['0' '>0']
DIF_SOCSTATUS_LAST_180DAY_G2  ['0' '>0']
DIF_TP_LAST_90DAY_G2  ['0' '>0']
DIF_3PESTEL_LAST_180DAY_G2  ['0' '>0']
DIF_INN_BY_CELLTEL_LAST_30_  ['0' '>0']


In [38]:
#1. change the data type
validation_df_clean['LOYALTY_STATUS'] = validation_df_clean['LOYALTY_STATUS'].astype(int)
validation_df_clean['CONTACT_IN_BL'] = validation_df_clean['CONTACT_IN_BL'].astype(int)

In [39]:
#2. change the values
for col in validation_binary_class_cols[2:]:
    validation_df_clean[col] = validation_df_clean[col].apply(lambda x: 0 if x=='0' else 1)

In [443]:
# 3. one hot vectors
validation_other_class_cols = [c for c in validation_columns_list if c not in validation_binary_class_cols]
validation_other_class_cols = validation_other_class_cols[1:-4]

for i, col in enumerate(validation_other_class_cols):
    if i == 0:
        validation_id_cols = pd.get_dummies(validation_df_clean[col], prefix = col).astype(int)
    else:
        validation_id_cols = id_cols.join(pd.get_dummies(validation_df_clean[col], prefix = col).astype(int))

# join the table
validation_df_clean = validation_df_clean.join(validation_id_cols).drop(validation_other_class_cols, axis=1)
print(len(validation_df_clean.columns))

#### 检查训练数据和评估数据是否有相同的变量
发现多余的变量均是异常值（统计数目为1条数据），于是进行简易删除

In [51]:
for a in id_cols.columns:
    if a not in validation_id_cols.columns:
        print(a)
print('***')
for a in validation_id_cols.columns:
    if a not in id_cols.columns:
        print(a)

HOUR_1
APPWPERIOD_R2_.5
DIF_INN_BY_CELLTEL_ALLPERIOD_7.0
***
DIF_INN_BY_CELLTEL_ALLPERIOD_4.0


In [59]:
# deal with some missing outliers & not neccesary columns
df_clean = df_clean.drop(['HOUR_1', 'APPWPERIOD_R2_.5', 'DIF_INN_BY_CELLTEL_ALLPERIOD_7.0'], axis = 1)
validation_df_clean = validation_df_clean.drop(['DIF_INN_BY_CELLTEL_ALLPERIOD_4.0'], axis = 1)

#### 对结果进行备份

In [61]:
df_clean.to_csv('cleaned_development_sample.csv')
validation_df_clean.to_csv('cleaned_assessment_sample.csv')

<a name="resample"></a>
# STEP 3. 数据重新采样

In [3]:
df_clean = pd.read_csv('cleaned_development_sample.csv').drop('Unnamed: 0', axis = 1)

In [4]:
print(Counter(df_clean['TARGET']).items())

dict_items([(0, 292497), (1, 5398)])


### 处理思路：
由上可知，在训练样本中，标签为 `1（bad）` 的数据仅有5000多条，不到所有数据的 2%。样本分布非常不均衡，如果直接使用该样本训练模型，最终可能导致模型将所有数据都分为 `0`，这样虽然可以达到 98% 的正确率，但是并不是我们想要的结果。对数据分布不均衡可以有如下处理方式：
> 1. 向下（欠）采样：最简单的方式是对数据较少的类进行随机放回抽样，属于非启发式方法。启发式算法有Tomek links、NearMiss
> 2. 向上（过）采样：最简单的方式是对数据较多的类进行随机不放回抽样，也属于非启发式方法。启发式算法有SMOTE
> 3. 综合采样：启发式算法有SMOTE+Tomek links、SMOTE+ENN

##### python的实现：
> * 对于非启发式的方法，python中使用 `sklearn.utils.resample` 可以实现
> * 启发式的算法，使用 `imbalanced-learn` 可以实现（http://contrib.scikit-learn.org/imbalanced-learn/stable/install.html）

### 最终采取方式：
> * 我先使用SOMTE+ENN算法（根据官方介绍文档，该算法相比SMOTE+Tomek Links更能消除噪音），在重新采样之后，新的Development data变成了`X_resampled`和`y_resampled`，不过label中 1 和 0 的比例大致变成 3：1（ \[(0, 78584), (1, 239038)\] ），这个非常奇怪。之后试图使用down-sampling的方法，但是运行速度非常慢，效果也不理想。
> * 最终选择非启发式的方法，将数据比例调整成为1：3。但是这个方法有个很大的缺点： <FONT COLOR = 'RED'>因为对少数类样本进行放回抽样，就算做了交叉检验也会出现最终得到估算的正确率比实际高的情况。</FONT>

#### （问题）
> * 1. 应该采取什么方式（启发式/非启发式）平衡数据？
> * 2. 采样应该在数据变量预处理（指缺失值、异常值、变量标准化）之前还是之后？
> * 3. 数据不同类别的比例应该为多少比较合适？具体数值应该为多少比较合适？（例如在这个问题里，标签为 `1` 的数据只有5000多条，如果把 `0` 的数据减少到 5000~50000 条，会损失很多训练数据；但如果把 `1` 重复抽样到 50000 条，会出现很多重复数据）
> * 4. 在反欺诈问题中，是否允许大量标签为 `1` 的重复样本多次出现的情况？

In [38]:
X = df_clean.drop('TARGET', axis = 1)
y = df_clean['TARGET']

In [91]:
# SMOTE + ENN
# The result seems not very good
smote_enn = SMOTEENN(random_state = 123)
X_smoteenn, y_smoteenn = smote_enn.fit_sample(X, y)
print(sorted(Counter(y_smoteenn).items()))

[(0, 78584), (1, 239038)]


In [6]:
# Random Select

# Separate majority and minority classes
df_majority = df_clean[df_clean.TARGET==0]
df_minority = df_clean[df_clean.TARGET==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=50000,    # to match majority class
                                 random_state=123) # reproducible results

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=150000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_up_down_sampled = pd.concat([df_majority_downsampled, df_minority_upsampled])

# Get the X, y values
X_resampled = df_up_down_sampled.drop('TARGET', axis = 1)
y_resampled = df_up_down_sampled['TARGET']
print(sorted(Counter(y_resampled).items()))

[(0, 150000), (1, 50000)]


<a name="features"></a>
# STEP 4. 特征选择 & 特征提取

### 处理思路：
因为该数据几乎全部变量都为分类变量，我不是很清楚使用PCA来进行特征提取（需要变量经过标准化处理且为正态分布）或者使用线性相关分析等方式进行特征选择是否能得出比较好的结果。所以在这一部分，我选择不对变量进行选择或者提取，在具体建模的时候通过改变正则化的惩罚值来对变量进行筛除。

<a name="models"></a>
# STEP 5. 建模
### 处理思路：
1. 不能使用 [K-NN](#knn) 模型，因为原数据不平衡，在随机综合采样之后，把标签为 `1` 的数据人为重复了多次，K-NN结果会没有意义
2. 以下尝试使用 [逻辑回归](#lr) 、[支持向量机](#svm) 和 [随机森林](#rf)
3. 使用交叉检验的方式选出最合理的参数

#### （问题）
1. 对于反欺诈类型的数据问题应该选择什么样的模型比较好？

<a name="lr"></a>
# STEP 5-1. Logistic Regression

### 处理思路：
> 1. 使用 `GridSearchCV` 调整参数，对regularization's penalty value（即 `C` 值）进行选择和交叉检验（5-fold cross validationa），评分为’average_precision‘
> 2. 选出最佳模型的参数，建模

### 结果：
> * 尝试了不同的数据采样比例和采样方式，发现Logistic Regression还是不能识别出 label = 1 的变量。

In [8]:
from sklearn.linear_model.logistic import LogisticRegression

In [9]:
# 调整参数
# define the parameter values that should be searched
param_grid = {"C": [0.01, 0.05, 0.1, 0.5, 1]}
lr = LogisticRegression()

# instantiate the grid
# AP summarizes a precision-recall curve as the weighted mean of precisions achieved at each threshold
# with the increase in recall from the previous threshold used as the weight
grid = GridSearchCV(lr, param_grid, cv = 5, scoring = 'average_precision')

# fit the grid with data
grid.fit(X_resampled, y_resampled)

# examine the best model
print(grid.best_score_)
print(grid.best_params_) 
print(grid.best_estimator_)

0.2505840854780201
{'C': 0.01}
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [9]:
# fit the model with the best params
lr = LogisticRegression(C = 0.01)
clf_lr= lr.fit(X_resampled, y_resampled)
pred_y = clf_lr.predict(X_resampled)

# get the detailed information
print('预测类：', np.unique(pred_y))
print('准确率：', accuracy_score(y_resampled, pred_y))  
print('精确率：', precision_score(y_resampled, pred_y))  
print('召回率：', recall_score(y_resampled, pred_y))  

预测类： [0]
准确率： 0.75
精确率： 0.0
召回率： 0.0


/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


<a name="svm"></a>
# STEP 5-2. SVM

### 处理思路：
> 1. 使用 `GridSearchCV` 调整参数，对 `C` 值和 `kernel`、`class_weight` 进行选择和交叉检验（5-fold cross validationa），评分为’average_precision‘
> 2. 选出最佳模型的参数，建模

### 结果：
> * SVC速度很慢，跑了一个多小时都没能出结果，所以以下仅为代码，并没有实际运行的结果。

In [9]:
from sklearn.svm import SVC 

In [ ]:
# define the parameter values that should be searched
param_grid = {"C":[0.1, 0.5, 1], 
              "kernel":['rbf','linear'], 
              "class_weight":['', 'balanced']}
svc = SVC()

# instantiate the grid
grid = GridSearchCV(svc, param_grid, cv = 5, scoring = 'average_precision')

# fit the grid with data
grid.fit(X_resampled, y_resampled)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)  
print(grid.best_estimator_)

In [ ]:
# fit the model with the best params
svc = SVC(C = 1, kernel = 'linear')
clf_svc = svc.fit(X_resampled, y_resampled)
pred_y = clf_svc.predict(X_resampled)

# get the detailed information
print('预测类：', np.unique(pred_y))
print('准确率：', accuracy_score(y_resampled, pred_y))  
print('精确率：', precision_score(y_resampled, pred_y))  
print('召回率：', recall_score(y_resampled, pred_y)) 

<a name="rf"></a>
# STEP 5-3. Random Forests

### 处理思路：
> 1. 使用 `GridSearchCV` 调整参数，对 `max_depth` 和 `n_estimators` 进行选择和交叉检验（5-fold cross validationa），评分为’average_precision‘
> 2. 选出最佳模型的参数，建模
> 3. 经查阅，决策树往往在类别不均衡数据上表现不错，随机森林不太会被数据不平衡影响（有待进一步学习），所以直接使用原始 X 和 y。

### 结果：
> * 选择出来`max_depth` 最佳值为10， `n_estimators` 最佳值为30。以此建模

#### （问题）
> * 在CV下选择出来的参数建模的召回率很低，但是反欺诈数据分析应该更在乎召回率？如果仅使用 `n_estimators` = 30 来建模虽然召回率和精确率都很高，但是不对树的深度进行规范，是否会出现过拟合的现象（尽管随机森林不容易出现过拟合）？

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
# define the parameter values that should be searched
param_grid = {"max_depth": [None, 10, 15], 
              "n_estimators":[10, 20, 30]}
rf = RandomForestClassifier(random_state=10)

# instantiate the grid
grid = GridSearchCV(rf, param_grid, cv = 5, scoring = 'average_precision')

# fit the grid with data
grid.fit(X, y)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)  
print(grid.best_estimator_)

0.06222975906916061
{'max_depth': 15, 'n_estimators': 30}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=10, verbose=0, warm_start=False)


In [49]:
rf = RandomForestClassifier(n_estimators=30)
clf_rf = rf.fit(X, y)
pred_y = clf_rf.predict(X)

# get the detailed information
print('预测类：', np.unique(pred_y))
print('准确率：', accuracy_score(y, pred_y))  
print('精确率：', precision_score(y, pred_y))  
print('召回率：', recall_score(y, pred_y)) 

预测类： [0 1]
准确率： 0.9989392235519227
精确率： 1.0
召回率： 0.9414597999258985


In [48]:
rf = RandomForestClassifier(n_estimators=30, max_depth=15)
clf_rf = rf.fit(X, y)
pred_y = clf_rf.predict(X)

# get the detailed information
print('预测类：', np.unique(pred_y))
print('准确率：', accuracy_score(y, pred_y))  
print('精确率：', precision_score(y, pred_y))  
print('召回率：', recall_score(y, pred_y)) 

预测类： [0 1]
准确率： 0.982970509743366
精确率： 1.0
召回率： 0.060207484253427195


# 其他：K-NN

### 思路：
为了检验K-NN确实不是有用的模型，我稍微做了一下尝试。最终结果，CV选择的 `n_neighbors` 参数为10, 大概正好符合随机抽取的标签为 `1` 的样本为原来样本10倍这一情况。该模型不可用。

In [101]:
from sklearn.neighbors import KNeighborsClassifier

In [105]:
# define the parameter values that should be searched
param_grid = {"n_neighbors": list(range(10, 20))}
knn = KNeighborsClassifier()

# instantiate the grid
grid = GridSearchCV(knn, param_grid, cv = 3, scoring = 'average_precision')

# fit the grid with data
grid.fit(X_resampled, y_resampled)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)  
print(grid.best_estimator_)

0.665228293552391
{'n_neighbors': 10}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')


In [107]:
# fit the model with the best params
knn = KNeighborsClassifier(n_neighbors = 10)
clf_knn = knn.fit(X_resampled, y_resampled)
pred_y = clf_knn.predict(X_resampled)

# get the detailed information
print('预测类：', np.unique(pred_y))
print('准确率：', accuracy_score(y_resampled, pred_y))  
print('精确率：', precision_score(y_resampled, pred_y))  
print('召回率：', recall_score(y_resampled, pred_y)) 

预测类： [0 1]
准确率： 0.903695
精确率： 0.7341376841399693
召回率： 0.96382


<a name="selection"></a>
# STEP 6. 模型选择

最终综合了多个模型和结果，选择 Random Forests 进行预测。

<a name="prediction"></a>
# STEP 7. 预测
用之前建的模型对 development data 和 assessment data 进行预测

In [54]:
# development data
prob_y = clf_rf.predict_proba(X)
prob_y_1 = [x[1] for x in prob_y]
df_clean = df_clean.assign(predicted_probability = prob_y_1)

In [63]:
# assessment data
validation_prob_y = clf_rf.predict_proba(validation_df_clean)
validation_prob_y_1 = [x[1] for x in validation_prob_y]
validation_df_clean = validation_df_clean.assign(predicted_probability  = validation_prob_y_1)

In [68]:
# save the results
df_clean.to_csv('development_sample_with_prob.csv')
validation_df_clean.to_csv('assessment_sample_with_prob.csv')